# Tools - Edge Seal Oxygen Ingress Calculator tool

### Calculation of oxygen ingress profile through an edge seal and into the encapsulant.

**Requirements:**
- Local weather data file or site longitude and latittude.
- Properties and dimensions of the edge seal.

**Objectives:**
1. Import weather data.
2. Set up the calculations.
3. Calculate oxygen ingress into an edge seal.
3. Incorporate an oxygen consumption model.
4. Plot the data.

**Background:**

This performs a 1-D finite difference model for oxygen ingress through an edge seal and into an encapsulant. This is effectively an infinitely long module with a prescribed width.The output is then displayed graphically.

In [1]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.3.3

In [53]:
import os
from pathlib import Path
import pvdeg
import pandas as pd
from pvdeg import DATA_DIR
#import dask
#import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns
#import math
import json

## 1. Import Weather Data

The function has these minimum requirements when using a weather data file:
- Weather data containing (at least) DNI, DHI, GHI, Temperature, RH, and Wind-Speed data at module level.
- Site meta-data containing (at least) latitude, longitude, and time zone

Alternatively one may can get meterological data from the NSRDB or PVGIS with just the longitude and latitude. This function for the NSRDB (via NSRDB 'PSM3') works primarily for most of North America and South America. PVGIS works for most of the rest of the world (via SARAH 'PVGIS'). See the tutorial "Weather Database Access.ipynb" tutorial on PVdeg or Jensen et al. https://doi.org/10.1016/j.solener.2023.112092 for satellite coverage information.

In [54]:
# Get data from a supplied data file (Do not use the next box of code if using your own file)
weather_file = os.path.join(DATA_DIR, 'psm3_demo.csv')
weather_df, meta = pvdeg.weather.read(weather_file,'csv')
print(meta)

{'Source': 'NSRDB', 'Location ID': 145809.0, 'City': '-', 'State': '-', 'Country': '-', 'Clearsky DHI Units': 'w/m2', 'Clearsky DNI Units': 'w/m2', 'Clearsky GHI Units': 'w/m2', 'Dew Point Units': 'c', 'DHI Units': 'w/m2', 'DNI Units': 'w/m2', 'GHI Units': 'w/m2', 'Solar Zenith Angle Units': 'Degree', 'Temperature Units': 'c', 'Pressure Units': 'mbar', 'Relative Humidity Units': '%', 'Precipitable Water Units': 'cm', 'Wind Direction Units': 'Degrees', 'Wind Speed Units': 'm/s', 'Cloud Type -15': 'N/A', 'Cloud Type 0': 'Clear', 'Cloud Type 1': 'Probably Clear', 'Cloud Type 2': 'Fog', 'Cloud Type 3': 'Water', 'Cloud Type 4': 'Super-Cooled Water', 'Cloud Type 5': 'Mixed', 'Cloud Type 6': 'Opaque Ice', 'Cloud Type 7': 'Cirrus', 'Cloud Type 8': 'Overlapping', 'Cloud Type 9': 'Overshooting', 'Cloud Type 10': 'Unknown', 'Cloud Type 11': 'Dust', 'Cloud Type 12': 'Smoke', 'Fill Flag 0': 'N/A', 'Fill Flag 1': 'Missing Image', 'Fill Flag 2': 'Low Irradiance', 'Fill Flag 3': 'Exceeds Clearsky', 'F

In [62]:
# This routine will get a meteorological dataset from anywhere in the world where it is available 
#weather_id = (24.7136, 46.6753) #Riyadh, Saudi Arabia
#weather_id = (35.6754, 139.65) #Tokyo, Japan
#weather_id = (-43.52646, 172.62165) #Christchurch, New Zealand
#weather_id = (64.84031, -147.73836) #Fairbanks, Alaska
#weather_id = (65.14037, -21.91633) #Reykjavik, Iceland
weather_id = (33.4152, -111.8315) #Mesa, Arizona
weather_df, meta = pvdeg.weather.get_anywhere(id=weather_id)
print(meta)

{'latitude': 33.4152, 'longitude': -111.8315, 'altitude': 381.0, 'wind_height': 10, 'Source': 'PVGIS'}


In [66]:
#This computes a module temperature. Here the default is an open rack system, but other options include: 
#       'open_rack_glass_glass', 
#       'close_mount_glass_glass',
#       'insulated_back_glass_polymer'

temperature = pvdeg.temperature.module(
    weather_df,
    meta,
    temp_model="sapm",
    conf="open_rack_glass_polymer",
    )
temperature = pd.DataFrame(temperature, columns = ['module_temperature'])

print(" ")
print(temperature)
print(weather_df)


The array tilt angle was not provided, therefore the latitude tilt of 33.4 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
 
                           module_temp
time(UTC)                             
2015-01-01 00:00:00+00:00     7.716772
2015-01-01 01:00:00+00:00     7.140000
2015-01-01 02:00:00+00:00     6.790000
2015-01-01 03:00:00+00:00     6.450000
2015-01-01 04:00:00+00:00     6.110000
...                                ...
2012-12-31 19:00:00+00:00    32.592905
2012-12-31 20:00:00+00:00    32.368340
2012-12-31 21:00:00+00:00    30.076761
2012-12-31 22:00:00+00:00    25.749276
2012-12-31 23:00:00+00:00    19.636465

[8760 rows x 1 columns]
                           temp_air  relative_humidity    ghi     dni   dhi  \
time(UTC)                                                                     
2015-01-01 00:00:00+00:00      7.48              66.54   10.0    0.00  10.0   
2015-01-01 01:00:00+00:00      7.14              67.89    0.0    0.0

## 2. Set up the Calculations

There is a library of some materials and the relevant oxygen ingress parameters that can be used.

In [42]:
es = 'OX005'    #This is the number for the edge seal in the json file
enc = 'OX003'   #This is the number for the encapsulant in the json file
esw = 1.5   #This is the edge seal width in [cm]
encw = 10   #This is the encapsulant width in [cm]
sn = 20     #This is the number of edge seal nodes to use
en = 50     #This is the number of encapsulant nodes to use

{'Name': 'Helioseal_101_dry', 'alias': 'PIB-A', 'Fickian': True, 'Ead': 45.01267147, 'Do': 3.370970011, 'Eas': -3.528965263, 'So': 5.25558996546881e-05, 'Eap': 41.48370621, 'Po': 1530700.085}
{'Name': 'EVA', 'alias': 'EVA', 'Fickian': True, 'Ead': 29.43112031, 'Do': 0.129061678, 'Eas': 32.3137806, 'So': 87.81142774, 'Eap': 61.7449009, 'Po': 97917899126}


## 3. Run the Calculations

This runs the calculations for diffusion using a simple 1-D finite difference calculation. The first set of calculations is just for diffusion, then the next two include some consumption of oxygen. In typical PV applications, it is common for oxygen ingress distance to be limited by its consumption rate in the encapsulant.

In [52]:
oxygen_profile = esdiffusion (temperature=temperature, es=es, enc=enc, esw=esw, encw=encw, sn=sn, en=en)


SyntaxError: invalid syntax (4150666294.py, line 3)